--- Tweeter parser ---

In [341]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'r8BGSocOtv6vZBnjcwkU5qM4u'
consumer_secret = 'CNAJg23IWXLvUUyJfuFUQOYfzvtKfQVQu6SsY82yJY7GGdhBDC'
access_token = '796083347315367936-grZPgV2cW8DvrozevvihPIJ1Oq9WQVK'
access_secret = 'oJoGFZYDDOXjyG30v0FBChWmGfF9RG6fXvDCVuNrDZJGb'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [342]:
#def process_or_store(tweet):
#    print(json.dumps(tweet))

In [343]:
#import json
#for tweet in tweepy.Cursor(api.user_timeline).items():
#    process_or_store(tweet._json)

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('negative.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

#--- filter for positive tweets 
#twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p'], languages=['fr'])

#--- filter for negative tweets
twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("], languages=['fr'])

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('positive.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

#--- filter for positive tweets 
twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p'], languages=['fr'])

#--- filter for negative tweets
#twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("], languages=['fr'])

--- Preprocessing part ---

In [269]:
from nltk.corpus import stopwords
import string

smiles_negativ = [':(', ':-(', '=(', ')-:', '):', ');', ')=']
smiles_positiv = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p']
pronoms = ['je', 'tu', 'nous', 'vous', 'il', 'elle', 'ils', 'elles', 'ça', '️']
smiles = smiles_negativ + smiles_positiv
punctuation = list(string.punctuation)
stop = stopwords.words('french') + punctuation + smiles + ['rt', 'les', '…', 'é', 'ca', 'ée', 'cette', '’', 'a', 'leurs', ''] + pronoms

In [270]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str1 = [
    r"(?:[\w_]+\s+pas)", # concat "pas" with the preceding word
    r"(?:pas+\s+[\w_]+[\w_]+[\w_]+)",
    r"(?:[a-z][a-z\-_]+[a-z])", # words with - 
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

regex_str2 = [
    r"(?:pas+\s+[\w_]+[\w_]+[\w_]+)",
    r'(?<=pas\s)(?<=\s)*très\s+[\w_]+', # concat "très" + word if there is no "pas" behind
    r'(?<=pas\s)(?<=\s)*trop\s+[\w_]+', # "trop" + word if there is no "pas" behind] # concat "pas" with the following word
    r'(?<=pas\s)(?<=\s)*moins\s+[\w_]+'
    ]

regex_str3 = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
]

regex_str4 = [
    r'(?<=pas\s)(?<=\s)*[\w_]+',# looks for the word that follows "pas " (to eliminate it afterwards)
    r'(?<=trop\s)(?<=\s)*[\w_]+',
    r'(?<=très\s)(?<=\s)*[\w_]+',
    r'(?<=moins\s)(?<=\s)*[\w_]+',
    ] 

tokens_re1 = re.compile(r'('+'|'.join(regex_str1)+')', re.VERBOSE | re.IGNORECASE)
tokens_re2 = re.compile(r'('+'|'.join(regex_str2)+')', re.VERBOSE | re.IGNORECASE)
tokens_re3 = re.compile(r'('+'|'.join(regex_str3)+')', re.VERBOSE | re.IGNORECASE)
tokens_re4 = re.compile(r'('+'|'.join(regex_str4)+')', re.VERBOSE | re.IGNORECASE)
 

In [271]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer

def stem_tokens(tokens):
    stemmer = FrenchStemmer()
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def preprocess(s):
    ### HTML tags, @-mentions, # hash-tags, # URLs, # numbers elimination
    s2 = re.sub(tokens_re3, "",s)
    ### simple tokinization (default function)
    tokens_init = word_tokenize(s2)
    ### stemming
    tokens_init = stem_tokens(tokens_init)
    ### string recreation
    s2 = "".join([" "+i for i in tokens_init]).strip()
    ### finding all non matching combinations from tokens_re1 in s2
    tokens1 = tokens_re1.findall(s2)
    ### ...from tokens_re4 in s2
    tokens4 = tokens_re4.findall(s2)
    ### cleaning tokens1 from stop-words and tokens4
    tokens1 = [term for term in tokens1 if term not in stop and term not in tokens4]
    ### "pas" + word
    tokens2 = tokens_re2.findall(s2)
    
    tokens = tokens1 + tokens2
    return tokens

In [371]:
tweet = "RT C'est pas ici @hoho: ne t'aime pas ! pas trop intelligente je ne veux pas y aller :D http://hoho.com #NLP"
preprocess(tweet)

['est pas', 'aim pas', 'pas trop', 'veux pas', 'aller', 'pas ici', 'pas trop']

--- Tweets' extracting from the json file ---

In [359]:
import json
import operator
from nltk import bigrams
from collections import Counter

tweets_text = []
mood = []
count_all = Counter()

for line in open('positive.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(1)
    count_all.update(preprocess(text))
  except:
    pass

In [360]:
import json
import operator
from nltk import bigrams
from collections import Counter

for line in open('negative.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(0)
    count_all.update(preprocess(text))
  except:
    pass

In [361]:
#count_all.most_common(2000)
less_frequent = count_all
for word in list(less_frequent):
    if less_frequent[word] > 5:
        del less_frequent[word]

--- Tf-Idf matrix ---

In [362]:
from sklearn.feature_extraction.text import TfidfVectorizer
stops = less_frequent.keys()
tfidf = TfidfVectorizer(tokenizer=preprocess, stop_words = stops)
A = tfidf.fit_transform(tweets_text)
A = A.toarray()

In [363]:
feature_names = tfidf.get_feature_names()

In [364]:
import numpy as np
A.shape

(8544, 1470)

In [365]:
#U, S, V = np.linalg.svd(A)
#print (U.shape)
#print (S.shape)
#print (V.shape)

In [366]:
#N = 500
#M = dot(dot(U[:,:N],diag(S)[:N,:N]),V[:N,:])

In [367]:
#V2 = V[:500,:]
#M = np.dot(A,V2.transpose())
M = A
M.shape

(8544, 1470)

--- Algorithmic testing part ---

In [368]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.cross_validation import train_test_split

X = M
Y = mood

X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [369]:
model = rf()
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred = np.round(pred)

In [370]:
np.sum(np.abs(y_test - pred))/len(y_test)

0.24485018726591759